<a href="https://colab.research.google.com/github/Abhinav-meluha/Waste-Segregation/blob/main/Waste%20Segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries (run once)
!pip install streamlit torch torchvision tensorflow pillow opencv-python

# Imports
import streamlit as st
import torch
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import cv2
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.6 MB/s eta 0:00:00


In [ ]:
# Install ultralytics for YOLOv5
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.7 MB/s eta 0:00:00


In [ ]:
# Load YOLOv5 model from PyTorch Hub
@st.cache_resource
def load_yolo_model():
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

yolo_model = load_yolo_model()

# Load MobileNetV2 for classification
@st.cache_resource
def load_mobilenet_model():
    model = MobileNetV2(weights='imagenet')
    return model

mobilenet_model = load_mobilenet_model()


2025-10-31 10:15:10.676 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:10.856 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-31 10:15:10.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:10.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:10.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-10-31 Python-3.12.12 torch-2.8.0+cu126 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 122MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
2025-10-31 10:15:34.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:34.208 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:34.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:34.213 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:34.215 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:34.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:34.

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


2025-10-31 10:15:35.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:35.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 10:15:35.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
def recommend_bin(label):
    label = label.lower()
    if any(word in label for word in ['food', 'banana', 'apple', 'vegetable', 'paper']):
        return "🟢 Biodegradable (Green Bin)"
    elif any(word in label for word in ['plastic', 'glass', 'metal', 'can', 'bottle']):
        return "🔵 Recyclable (Blue Bin)"
    elif any(word in label for word in ['battery', 'chemical', 'paint', 'medical']):
        return "🔴 Hazardous (Red Bin)"
    else:
        return "⚪ General Waste (Grey Bin)"


In [ ]:
!pip install tensorflow torch torchvision opencv-python pillow matplotlib numpy scikit-learn streamlit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# 📁 Path to your dataset folder in Drive
base_dir = "/content/drive/MyDrive/Waste Segregation System for smart cities"

print("✅ Dataset folder found!")
print("📂 Subfolders:", os.listdir(base_dir))


✅ Dataset folder found!
📂 Subfolders: ['Hazardous ', 'Recyclable', 'Biodegradable ']


In [ ]:
for root, dirs, files in os.walk(base_dir):
    print(root, "->", len(files), "files")


/content/drive/MyDrive/Waste Segregation System for smart cities -> 0 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous  -> 0 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous /battery -> 282 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous /trash -> 192 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable -> 0 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/cardboard -> 261 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/metal -> 279 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/glass -> 426 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/clothes -> 431 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/paper -> 278 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/s

In [ ]:
import os, shutil, random

source_dir = base_dir
target_dir = "/content/dataset_split"
os.makedirs(target_dir, exist_ok=True)

# Get the actual class directories, ignoring any files at the base_dir level
class_dirs = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

for cls in class_dirs:
    class_path = os.path.join(source_dir, cls)

    images = os.listdir(class_path)
    # Filter out any subdirectories within the class directory (like 'trash' in Hazardous)
    images = [img for img in images if os.path.isfile(os.path.join(class_path, img))]
    random.shuffle(images)

    n_total = len(images)
    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)
    n_test = n_total - n_train - n_val

    splits = {
        "train": images[:n_train],
        "val": images[n_train:n_train+n_val],
        "test": images[n_train+n_val:]
    }

    for split_name, split_imgs in splits.items():
        split_folder = os.path.join(target_dir, split_name, cls)
        os.makedirs(split_folder, exist_ok=True)
        for img_name in split_imgs:
            src = os.path.join(class_path, img_name)
            dst = os.path.join(split_folder, img_name)
            shutil.copy(src, dst)

print("✅ Dataset organized into train/val/test at:", target_dir)

✅ Dataset organized into train/val/test at: /content/dataset_split


In [ ]:
import os

real_base = "/content/drive/MyDrive/Waste Segregation System for smart cities"  # your dataset folder
for root, dirs, files in os.walk(real_base):
    print(root, "->", len(files), "files")


/content/drive/MyDrive/Waste Segregation System for smart cities -> 0 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous  -> 0 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous /battery -> 282 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous /trash -> 192 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable -> 0 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/cardboard -> 261 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/metal -> 279 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/glass -> 426 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/clothes -> 431 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/paper -> 278 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/s

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(target_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(target_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(target_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 2638 images belonging to 14 classes.
Found 853 images belonging to 14 classes.
Found 887 images belonging to 14 classes.


In [ ]:
import os

# Check inside your original Drive dataset folder
real_base = "/content/drive/MyDrive/Waste Segregation System for smart cities"  # 👈 change if name differs
for root, dirs, files in os.walk(real_base):
    if files:
        print(root, "->", len(files), "files")


/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous /battery -> 282 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous /trash -> 192 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/cardboard -> 261 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/metal -> 279 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/glass -> 426 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/clothes -> 431 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/paper -> 278 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/shoes -> 304 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/plastic -> 126 files
/content/drive/MyDrive/Waste Segregation System for smart cities/Biodegradable /biological -> 240 files


In [ ]:
import os, shutil, random

# ✅ Correct dataset path
source_dir = "/content/drive/MyDrive/Waste Segregation System for smart cities"
target_dir = "/content/dataset_split"
os.makedirs(target_dir, exist_ok=True)

# Automatically detect subfolders (classes)
classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
print("Detected classes:", classes)

for cls in classes:
    class_path = os.path.join(source_dir, cls)
    images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    n_total = len(images)
    if n_total == 0:
        print(f"⚠️ No images found in {cls}. Check folder spelling or contents.")
        continue

    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)
    n_test = n_total - n_train - n_val

    splits = {
        "train": images[:n_train],
        "val": images[n_train:n_train+n_val],
        "test": images[n_train+n_val:]
    }

    for split_name, split_imgs in splits.items():
        split_folder = os.path.join(target_dir, split_name, cls)
        os.makedirs(split_folder, exist_ok=True)
        for img_name in split_imgs:
            src = os.path.join(class_path, img_name)
            dst = os.path.join(split_folder, img_name)
            shutil.copy(src, dst)

print("✅ Dataset successfully split into train/val/test folders at:", target_dir)


Detected classes: ['Hazardous ', 'Recyclable', 'Biodegradable ']
⚠️ No images found in Hazardous . Check folder spelling or contents.
⚠️ No images found in Recyclable. Check folder spelling or contents.
⚠️ No images found in Biodegradable . Check folder spelling or contents.
✅ Dataset successfully split into train/val/test folders at: /content/dataset_split


In [ ]:
import os, shutil, random

source_dir = "/content/drive/MyDrive/Waste Segregation System for smart cities"
target_dir = "/content/dataset_split"
os.makedirs(target_dir, exist_ok=True)

# Get the main class directories (Hazardous , Recyclable, Biodegradable )
main_classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
print("Detected main classes:", main_classes)

# Dictionary to hold all image paths grouped by their original class
all_images_by_class = {}

# Traverse through subdirectories to find all image files
for main_cls in main_classes:
    main_cls_path = os.path.join(source_dir, main_cls)
    for root, dirs, files in os.walk(main_cls_path):
        # Get the class name from the subdirectory (e.g., 'battery', 'cardboard', etc.)
        # This assumes the immediate subdirectories within the main classes are the actual classes
        relative_path = os.path.relpath(root, main_cls_path)
        if relative_path == '.': # Skip the main class directory itself if it contains files
            continue

        class_name = os.path.basename(root)
        images = [os.path.join(root, f) for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        if class_name not in all_images_by_class:
            all_images_by_class[class_name] = []
        all_images_by_class[class_name].extend(images)

# Now split the images for each found class
print("Detected actual classes with images:", list(all_images_by_class.keys()))

for cls, image_paths in all_images_by_class.items():
    random.shuffle(image_paths)

    n_total = len(image_paths)
    if n_total == 0:
        print(f"⚠️ No images found for class {cls}.")
        continue

    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)
    n_test = n_total - n_train - n_val

    splits = {
        "train": image_paths[:n_train],
        "val": image_paths[n_train:n_train+n_val],
        "test": image_paths[n_train+n_val:]
    }

    for split_name, split_img_paths in splits.items():
        # Create target directories based on the actual class name
        split_folder = os.path.join(target_dir, split_name, cls)
        os.makedirs(split_folder, exist_ok=True)

        for img_src_path in split_img_paths:
            img_name = os.path.basename(img_src_path)
            img_dst_path = os.path.join(split_folder, img_name)
            shutil.copy(img_src_path, img_dst_path)

print("✅ Dataset successfully split into train/val/test folders at:", target_dir)

Detected main classes: ['Hazardous ', 'Recyclable', 'Biodegradable ']
Detected actual classes with images: ['battery', 'trash', 'cardboard', 'metal', 'glass', 'clothes', 'paper', 'shoes', 'plastic', 'biological']
✅ Dataset successfully split into train/val/test folders at: /content/dataset_split


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# ✅ Use your newly organized dataset
base_dir = "/content/dataset_split"
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 2763 images belonging to 14 classes.
Found 1128 images belonging to 14 classes.
Found 1190 images belonging to 14 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 pretrained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze base layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
# Update the output layer to match the number of classes detected by the data generator (14)
output = Dense(14, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,758 (9.25 MB)

 Trainable params: 165,774 (647.55 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from PIL import Image
import os

broken_files = []

for root, dirs, files in os.walk("/content/dataset_split"):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(root, file)
            try:
                img = Image.open(path)
                img.verify()  # verify image integrity
            except Exception as e:
                broken_files.append(path)

print(f"🔍 Found {len(broken_files)} corrupted or invalid image(s).")
if broken_files:
    print("\nExample corrupted files:")
    print("\n".join(broken_files[:10]))


🔍 Found 1 corrupted or invalid image(s).

Example corrupted files:
/content/dataset_split/train/metal/metal_1607.jpg


In [ ]:
for f in broken_files:
    os.remove(f)
print("✅ Removed all corrupted images successfully.")


✅ Removed all corrupted images successfully.


In [ ]:
import os

valid_ext = ('.jpg', '.jpeg', '.png')
removed = 0

for root, dirs, files in os.walk("/content/dataset_split"):
    for file in files:
        if not file.lower().endswith(valid_ext):
            os.remove(os.path.join(root, file))
            removed += 1

print(f"✅ Removed {removed} non-image files.")


✅ Removed 0 non-image files.


In [ ]:
train_gen = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    follow_links=False
)


Found 2762 images belonging to 14 classes.


In [ ]:
!ls /content/dataset_split/train/
!ls /content/dataset_split/train/metal | head


 battery	   cardboard  'Hazardous '   plastic	   shoes
'Biodegradable '   clothes     metal	     Recyclable    trash
 biological	   glass       paper	     Recycleable
20240626_161207.jpg
metal_1001.jpg
metal_1003.jpg
metal_1006.jpg
metal_1014.jpg
metal_1016.jpg
metal_1029.jpg
metal_1036.jpg
metal_1043.jpg
metal_1051.jpg


In [ ]:
import os

base_path = "/content/dataset_split/train"

# Walk through the directory tree from the bottom up
for root, dirs, files in os.walk(base_path, topdown=False):
    # Check if the current directory is empty (contains no files and no subdirectories listed by os.walk)
    if not files and not dirs:
        print(f"🗑️ Removing empty folder: {root}")
        os.rmdir(root)

🗑️ Removing empty folder: /content/dataset_split/train/Recyclable
🗑️ Removing empty folder: /content/dataset_split/train/Recycleable
🗑️ Removing empty folder: /content/dataset_split/train/Biodegradable 
🗑️ Removing empty folder: /content/dataset_split/train/Hazardous 


In [ ]:
for split in ['train', 'val', 'test']:
    path = f"/content/dataset_split/{split}"
    for root, dirs, files in os.walk(path):
        if len(files) == 0:
            print(f"⚠️ Warning: empty folder detected in {split}: {root}")


⚠️ Warning: empty folder detected in train: /content/dataset_split/train
⚠️ Warning: empty folder detected in val: /content/dataset_split/val
⚠️ Warning: empty folder detected in val: /content/dataset_split/val/Recyclable
⚠️ Warning: empty folder detected in val: /content/dataset_split/val/Recycleable
⚠️ Warning: empty folder detected in val: /content/dataset_split/val/Biodegradable 
⚠️ Warning: empty folder detected in val: /content/dataset_split/val/Hazardous 
⚠️ Warning: empty folder detected in test: /content/dataset_split/test
⚠️ Warning: empty folder detected in test: /content/dataset_split/test/Recyclable
⚠️ Warning: empty folder detected in test: /content/dataset_split/test/Recycleable
⚠️ Warning: empty folder detected in test: /content/dataset_split/test/Biodegradable 
⚠️ Warning: empty folder detected in test: /content/dataset_split/test/Hazardous 


In [ ]:
!ls "/content/drive/MyDrive/Waste Segregation System for smart cities"


'Biodegradable '  'Hazardous '	 Recyclable


In [ ]:
import os, shutil, random

source_dir = "/content/drive/MyDrive/Waste Segregation System for smart cities"
target_dir = "/content/dataset_split"
os.makedirs(target_dir, exist_ok=True)

# Recognize all common image extensions
valid_exts = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')

# Get the main class directories (Hazardous , Recyclable, Biodegradable )
main_classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
print("Detected main classes:", main_classes)

# Dictionary to hold all image paths grouped by their original class
all_images_by_class = {}

# Traverse through subdirectories to find all image files
for main_cls in main_classes:
    main_cls_path = os.path.join(source_dir, main_cls)
    for root, dirs, files in os.walk(main_cls_path):
        # Get the class name from the subdirectory (e.g., 'battery', 'cardboard', etc.)
        # This assumes the immediate subdirectories within the main classes are the actual classes
        relative_path = os.path.relpath(root, main_cls_path)
        if relative_path == '.': # Skip the main class directory itself if it contains files
            continue

        class_name = os.path.basename(root)
        images = [os.path.join(root, f) for f in files if f.lower().endswith(valid_exts)]

        if class_name not in all_images_by_class:
            all_images_by_class[class_name] = []
        all_images_by_class[class_name].extend(images)

# Now split the images for each found class
print("Detected actual classes with images:", list(all_images_by_class.keys()))

for cls, image_paths in all_images_by_class.items():
    random.shuffle(image_paths)

    n_total = len(image_paths)
    if n_total == 0:
        print(f"⚠️ No images found for class {cls}.")
        continue

    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)
    n_test = n_total - n_train - n_val

    splits = {
        "train": image_paths[:n_train],
        "val": image_paths[n_train:n_train+n_val],
        "test": image_paths[n_train+n_val:]
    }

    for split_name, split_img_paths in splits.items():
        # Create target directories based on the actual class name
        split_folder = os.path.join(target_dir, split_name, cls)
        os.makedirs(split_folder, exist_ok=True)

        for img_src_path in split_img_paths:
            img_name = os.path.basename(img_src_path)
            img_dst_path = os.path.join(split_folder, img_name)
            shutil.copy(img_src_path, img_dst_path)

print("✅ Split completed successfully!")

Detected main classes: ['Hazardous ', 'Recyclable', 'Biodegradable ']
Detected actual classes with images: ['battery', 'trash', 'cardboard', 'metal', 'glass', 'clothes', 'paper', 'shoes', 'plastic', 'biological']
✅ Split completed successfully!


In [ ]:
for root, dirs, files in os.walk("/content/dataset_split"):
    if len(files) > 0:
        print(root, "->", len(files), "files")


/content/dataset_split/train/battery -> 282 files
/content/dataset_split/train/cardboard -> 261 files
/content/dataset_split/train/shoes -> 300 files
/content/dataset_split/train/clothes -> 427 files
/content/dataset_split/train/paper -> 274 files
/content/dataset_split/train/biological -> 237 files
/content/dataset_split/train/plastic -> 126 files
/content/dataset_split/train/glass -> 419 files
/content/dataset_split/train/metal -> 279 files
/content/dataset_split/train/trash -> 192 files
/content/dataset_split/val/battery -> 149 files
/content/dataset_split/val/cardboard -> 144 files
/content/dataset_split/val/shoes -> 142 files
/content/dataset_split/val/clothes -> 202 files
/content/dataset_split/val/paper -> 134 files
/content/dataset_split/val/biological -> 119 files
/content/dataset_split/val/plastic -> 61 files
/content/dataset_split/val/glass -> 195 files
/content/dataset_split/val/metal -> 133 files
/content/dataset_split/val/trash -> 105 files
/content/dataset_split/test/bat

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

base_dir = "/content/dataset_split"
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=25, width_shift_range=0.2,
                                   height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(os.path.join(base_dir, 'train'),
                                              target_size=img_size, batch_size=batch_size,
                                              class_mode='categorical')

val_gen = val_datagen.flow_from_directory(os.path.join(base_dir, 'val'),
                                          target_size=img_size, batch_size=batch_size,
                                          class_mode='categorical')

test_gen = test_datagen.flow_from_directory(os.path.join(base_dir, 'test'),
                                            target_size=img_size, batch_size=batch_size,
                                            class_mode='categorical', shuffle=False)


Found 2797 images belonging to 10 classes.
Found 1384 images belonging to 14 classes.
Found 1447 images belonging to 14 classes.


In [ ]:
print("Train classes:", train_gen.class_indices)
print("Number of classes detected:", len(train_gen.class_indices))


Train classes: {'battery': 0, 'biological': 1, 'cardboard': 2, 'clothes': 3, 'glass': 4, 'metal': 5, 'paper': 6, 'plastic': 7, 'shoes': 8, 'trash': 9}
Number of classes detected: 10


In [ ]:
n_classes = len(train_gen.class_indices)
print("✅ Actual number of classes =", n_classes)



✅ Actual number of classes = 10


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(n_classes, activation='softmax')(x)  # ✅ dynamically set correct number

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_5[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
print("Train classes:", train_gen.class_indices)
print("Number of classes detected:", len(train_gen.class_indices))


Train classes: {'battery': 0, 'biological': 1, 'cardboard': 2, 'clothes': 3, 'glass': 4, 'metal': 5, 'paper': 6, 'plastic': 7, 'shoes': 8, 'trash': 9}
Number of classes detected: 10


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ✅ You have 10 classes
n_classes = 10

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze pretrained layers

# Build custom classifier head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)  # helps prevent overfitting
output = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Clean up previous models/sessions to prevent cached graph issues
tf.keras.backend.clear_session()


In [ ]:
base_dir = "/content/dataset_split"
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Detect number of classes automatically
n_classes = len(train_gen.class_indices)
print("✅ Number of detected classes:", n_classes)


Found 2797 images belonging to 10 classes.
Found 1384 images belonging to 14 classes.
Found 1447 images belonging to 14 classes.
✅ Number of detected classes: 10


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Build model dynamically
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(n_classes, activation='softmax')(x)  # ✅ uses dynamic class count

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# --- 1. Setup Placeholder Model and Data Generator ---

# A simple Keras model for demonstration
def build_model():
    """Defines a simple sequential model."""
    model = Sequential([
        Dense(10, activation='relu', input_shape=(20,)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# A placeholder generator function (simulating train_gen/val_gen)
# It is now truly infinite to support multiple epochs.
def data_generator(batch_size):
    """Generates random data batches for training infinitely."""
    while True:
        # Generate a batch of dummy data (20 features, 1 label)
        X = np.random.rand(batch_size, 20).astype('float32')
        y = np.random.randint(0, 2, size=(batch_size, 1)).astype('float32')
        yield X, y
        # Removed logic to break the loop, making it truly infinite.
        # The calling code (the custom loop) is responsible for step counting.

# --- 2. Training Configuration ---

model = build_model()
EPOCHS = 10
BATCH_SIZE = 32
STEPS_PER_EPOCH = 100
VAL_STEPS = 20

# Updated generator instantiation: removed the step limits since the function is now infinite.
train_gen = data_generator(BATCH_SIZE)
val_gen = data_generator(BATCH_SIZE)

print(f"Starting custom training loop for {EPOCHS} epochs...")
print("-" * 50)

# --- 3. The Custom Training Loop (The Alternate Code) ---

# Dictionary to store history, similar to model.fit() output
custom_history = {
    'loss': [],
    'accuracy': [],
    'val_loss': [],
    'val_accuracy': []
}

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")

    # Training Phase
    epoch_loss = 0
    epoch_acc = 0

    # Iterate through the training generator for a fixed number of steps
    for step in range(STEPS_PER_EPOCH):
        # Get the next batch
        x_batch, y_batch = next(train_gen)

        # Manually train on a single batch
        # This is the core alternative to model.fit()
        metrics = model.train_on_batch(x_batch, y_batch, return_dict=True)

        # Accumulate metrics
        epoch_loss += metrics['loss']
        epoch_acc += metrics['accuracy']

        # Print progress (optional)
        if (step + 1) % 10 == 0 or (step + 1) == STEPS_PER_EPOCH:
            avg_loss = epoch_loss / (step + 1)
            avg_acc = epoch_acc / (step + 1)
            print(f"Step {step+1}/{STEPS_PER_EPOCH} - loss: {avg_loss:.4f} - accuracy: {avg_acc:.4f}", end='\r')

    # Calculate average training metrics for the epoch
    avg_train_loss = epoch_loss / STEPS_PER_EPOCH
    avg_train_acc = epoch_acc / STEPS_PER_EPOCH

    # Validation Phase
    val_loss = 0
    val_acc = 0

    # Iterate through the validation generator for a fixed number of steps
    for step in range(VAL_STEPS):
        x_val, y_val = next(val_gen)
        # Manually evaluate on a single batch
        val_metrics = model.test_on_batch(x_val, y_val, return_dict=True)
        val_loss += val_metrics['loss']
        val_acc += val_metrics['accuracy']

    # Calculate average validation metrics
    avg_val_loss = val_loss / VAL_STEPS
    avg_val_acc = val_acc / VAL_STEPS

    # Log and print results
    custom_history['loss'].append(avg_train_loss)
    custom_history['accuracy'].append(avg_train_acc)
    custom_history['val_loss'].append(avg_val_loss)
    custom_history['val_accuracy'].append(avg_val_acc)

    print(f"Epoch {epoch}/{EPOCHS} - Loss: {avg_train_loss:.4f} - Acc: {avg_train_acc:.4f} - Val Loss: {avg_val_loss:.4f} - Val Acc: {avg_val_acc:.4f}")
    print("-" * 50)

print("Custom training complete.")
# The 'custom_history' dictionary now contains the training results.


Starting custom training loop for 10 epochs...
--------------------------------------------------
Epoch 1/10
Epoch 1/10 - Loss: 0.7057 - Acc: 0.4883 - Val Loss: 0.7010 - Val Acc: 0.5005
--------------------------------------------------
Epoch 2/10
Epoch 2/10 - Loss: 0.7007 - Acc: 0.5019 - Val Loss: 0.6998 - Val Acc: 0.5034
--------------------------------------------------
Epoch 3/10
Epoch 3/10 - Loss: 0.6992 - Acc: 0.5043 - Val Loss: 0.6992 - Val Acc: 0.5010
--------------------------------------------------
Epoch 4/10
Epoch 4/10 - Loss: 0.6989 - Acc: 0.4998 - Val Loss: 0.6986 - Val Acc: 0.4990
--------------------------------------------------
Epoch 5/10
Epoch 5/10 - Loss: 0.6984 - Acc: 0.4993 - Val Loss: 0.6981 - Val Acc: 0.4997
--------------------------------------------------
Epoch 6/10
Epoch 6/10 - Loss: 0.6978 - Acc: 0.5005 - Val Loss: 0.6977 - Val Acc: 0.5000
--------------------------------------------------
Epoch 7/10
Epoch 7/10 - Loss: 0.6975 - Acc: 0.4992 - Val Loss: 0.697

In [ ]:
# Unfreeze the last 30 layers for fine-tuning
for layer in model.layers[:-30]:
    layer.trainable = False
for layer in model.layers[-30:]:
    layer.trainable = True

# Compile again with a smaller learning rate
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # small LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221 (884.00 B)

 Trainable params: 221 (884.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Unfreeze the last 30 layers of MobileNetV2
for layer in model.layers[:-30]:
    layer.trainable = False
for layer in model.layers[-30:]:
    layer.trainable = True

from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221 (884.00 B)

 Trainable params: 221 (884.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# --- Diagnostic check ---
try:
    print("Type of history_ft:", type(history_ft))
    if hasattr(history_ft, "history"):
        print("Keys inside history_ft.history:", history_ft.history.keys())
    else:
        print("No attribute .history found in history_ft")
except NameError:
    print("Variable history_ft does not exist in memory")


Variable history_ft does not exist in memory


In [ ]:
import os

train_dir = "/content/drive/MyDrive/Waste Segregation System for smart cities"
for root, dirs, files in os.walk(train_dir):
    print(root, "→", len([f for f in files if f.lower().endswith(('.jpg','.png','.jpeg'))]), "images")


/content/drive/MyDrive/Waste Segregation System for smart cities → 0 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous → 0 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous/battery → 282 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Hazardous/trash → 192 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable → 0 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/cardboard → 261 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/metal → 279 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/glass → 426 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/clothes → 431 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/paper → 278 images
/content/drive/MyDrive/Waste Segregation System for smart cities/Recyclable/shoe

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

# Split automatically using validation_split
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

train_gen = datagen.flow_from_directory(
    "/content/drive/MyDrive/Waste Segregation System for smart cities",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    "/content/drive/MyDrive/Waste Segregation System for smart cities",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)


Found 2256 images belonging to 3 classes.
Found 563 images belonging to 3 classes.


In [ ]:
print("Classes:", train_gen.class_indices)


Classes: {'Biodegradable': 0, 'Hazardous': 1, 'Recyclable': 2}


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os

# Load MobileNetV2 base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
os.makedirs("/content/best_model", exist_ok=True)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ModelCheckpoint('/content/best_model/best_model.h5', save_best_only=True, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6, verbose=1)
]

# Train
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1,
    callbacks=callbacks
)


Epoch 1/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7726 - loss: 0.7008
Epoch 1: val_loss improved from inf to 0.29354, saving model to /content/best_model/best_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 196s 3s/step - accuracy: 0.7737 - loss: 0.6974 - val_accuracy: 0.8845 - val_loss: 0.2935 - learning_rate: 0.0010
Epoch 2/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9176 - loss: 0.2151
Epoch 2: val_loss improved from 0.29354 to 0.17760, saving model to /content/best_model/best_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 181s 3s/step - accuracy: 0.9177 - loss: 0.2149 - val_accuracy: 0.9378 - val_loss: 0.1776 - learning_rate: 0.0010
Epoch 3/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9448 - loss: 0.1608
Epoch 3: val_loss did not improve from 0.17760
71/71 ━━━━━━━━━━━━━━━━━━━━ 178s 3s/step - accuracy: 0.9447 - loss: 0.1609 - val_accuracy: 0.9361 - val_loss: 0.1777 - learning_rate: 0.0010
Epoch 4/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9577 - loss: 0.1135
Epoch 4: val_loss did not improve from 0.17760

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
71/71 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.9577 - loss: 0.1136 - val_accuracy: 0.9112 - val_loss: 0.2461 - learning_rate: 0.0010
Epoch 5/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9578 - loss: 0.1127
Epoch 5: val_loss did not improve from 0.17760
71/71 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - accuracy: 0.9579 - loss: 0.1125 - val_accuracy: 0.9290 - val_loss: 0.18

In [ ]:
loss, acc = model.evaluate(val_gen)
print(f"✅ Validation Accuracy: {acc*100:.2f}%")
print(f"Validation Loss: {loss:.4f}")


18/18 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.9281 - loss: 0.1856
✅ Validation Accuracy: 93.25%
Validation Loss: 0.1831


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
model.save('/content/final_waste_classifier.keras')
print("✅ Final model saved as 'final_waste_classifier.keras'")


✅ Final model saved as 'final_waste_classifier.keras'


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt

# Upload an image
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename
    img = image.load_img(img_path, target_size=(224,224))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = list(train_gen.class_indices.keys())[np.argmax(prediction)]
    print(f"🟢 Predicted Class: {predicted_class}")


Saving cardboard_12.jpg to cardboard_12.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
🟢 Predicted Class: Recyclable


In [ ]:
model.save('/content/drive/MyDrive/final_waste_classifier.keras')


In [ ]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Load the model
model = load_model('final_waste_classifier.keras')

# Class labels (same order as training)
class_labels = ['Biodegradable', 'Hazardous', 'Recyclable']

# Streamlit UI
st.title("♻️ Smart Waste Classifier")
st.write("Upload an image to classify it as **Biodegradable**, **Hazardous**, or **Recyclable**")

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)

    # Preprocess the image
    img = img.resize((224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)

    st.success(f"✅ Predicted Class: **{class_labels[predicted_class]}**")
    st.info(f"Confidence: {confidence*100:.2f}%")


Overwriting app.py


In [ ]:
!pip install pyngrok


In [11]:
!pip install cloudflared -q
!streamlit run app.py & sleep 5 && cloudflared tunnel --url http://localhost:8501 --no-autoupdate


Streankit
streamlit
https://xxxx-xx-xx-xx.ngrok-free.app
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.3 MB/s eta 0:00:00



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.231.77.112:8501

/bin/bash: line 1: cloudflared: command not found
  Stopping...


In [12]:
%%writefile app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image

# --- Load model ---
@st.cache_resource
def load_trained_model():
    return load_model("final_waste_classifier.keras")

model = load_trained_model()

# --- Class names (must match your training classes) ---
CLASS_NAMES = ['Biodegradable', 'Hazardous', 'Recyclable']

# --- Streamlit UI ---
st.set_page_config(page_title="Smart Waste Classifier", page_icon="♻️")
st.title("♻️ Smart Waste Classifier")
st.write("Upload an image to identify whether it’s **Biodegradable**, **Hazardous**, or **Recyclable**.")

uploaded_file = st.file_uploader("📤 Upload an Image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)

    # Preprocess the image
    img = img.resize((224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    preds = model.predict(img_array)
    predicted_class = CLASS_NAMES[np.argmax(preds)]
    confidence = np.max(preds) * 100

    # Display result
    st.success(f"🟢 **Prediction:** {predicted_class}")
    st.write(f"💡 Confidence: {confidence:.2f}%")

    # Show probabilities for each class
    st.subheader("Class Probabilities")
    st.bar_chart(
        {CLASS_NAMES[i]: float(preds[0][i]) for i in range(len(CLASS_NAMES))}
    )


Overwriting app.py


In [15]:
# Download and install cloudflared for a Debian/Ubuntu system
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!sudo mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

https://waste-management.ngrok-free.dev/
--2025-10-31 12:52:34--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.10.1/cloudflared-linux-amd64 [following]
--2025-10-31 12:52:34--  https://github.com/cloudflare/cloudflared/releases/download/2025.10.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/d78430f2-7684-4130-ab31-f00b6b0d15e8?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-31T13%3A39%3A41Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-9

In [ ]:
!streamlit run app.py & sleep 5 && cloudflared tunnel --url http://localhost:8501 --no-autoupdate




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.231.77.112:8501

2025-10-31T12:52:57Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-31T12:52:57Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-31T12:53:01Z INF +--------------------------------------------------------------------------------------------+
2025-10-31T12:53:01Z INF 